## Imports

In [18]:
import math
import numpy as np
import pandas
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score

# Preparando el dataset de entrenamiento

In [67]:
data_train = pandas.read_csv('./data/perrosTrainDataAll.csv')

data_train

,Mascota,Edad,Tamaño,Sexo,Patron de pelaje,Color de pelaje 1,Color de pelaje 2,Color de pelaje 3,Largo de pelaje,Color de ojos,Largo de hocico,Largo de cola,Largo de orejas,Tipo de orejas,Pata
0,1,Adulto,Mediano,NaN,Bicolor,Blanco,Negro,NaN,Corto,NaN,Corto,NaN,Cortas,Cuatro,NaN
1,2,Cachorro,Mediano,NaN,Bicolor,Blanco,Marron,NaN,NaN,NaN,Largo,NaN,Cortas,Caidas,NaN
2,3,Adulto,Mediano,NaN,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Largo,NaN,Largas,NaN,NaN
3,4,Adulto,Mediano,NaN,NaN,NaN,NaN,NaN,Corto,NaN,Largo,NaN,NaN,Paradas,NaN
4,5,NaN,Chico,NaN,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Corto,NaN,Cortas,Caidas,NaN
5,6,NaN,Chico,NaN,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Corto,NaN,Cortas,Caidas,NaN
6,7,NaN,Mediano,NaN,NaN,Negro,NaN,NaN,Corto,Marron oscuro,Corto,NaN,Cortas,NaN,NaN
7,8,Adulto,Mediano,NaN,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Largo,Largo,NaN,Caidas,NaN
8,9,Cachorro,Chico,NaN,Bicolor,Blanco,NaN,NaN,Corto,NaN,NaN,Corto,Cortas,Caidas,NaN
9,10,Adulto,Grande,NaN,Liso,NaN,NaN,NaN,Mediano,Marron oscuro,Largo,Largo,Largas,NaN,NaN


### Obtenemos la cantidad de valores vacios

In [20]:
null_value_stats = data_train.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Edad                  9
Sexo                 20
Patron de pelaje      8
Color de pelaje 1     7
Color de pelaje 2    21
Color de pelaje 3    25
Largo de pelaje       4
Color de ojos        10
Largo de hocico       4
Largo de cola         9
Largo de orejas       8
Tipo de orejas       11
dtype: int64

### Lleno los nulos con strings vacios y quito la columna "Mascota"

In [21]:
data_train.fillna("NaN", inplace=True)
prepared_data_train = data_train.drop('Mascota', axis=1)
prepared_data_train

,Edad,Tamaño,Sexo,Patron de pelaje,Color de pelaje 1,Color de pelaje 2,Color de pelaje 3,Largo de pelaje,Color de ojos,Largo de hocico,Largo de cola,Largo de orejas,Tipo de orejas
0,Adulto,Mediano,NaN,Bicolor,Blanco,Negro,NaN,Corto,NaN,Corto,NaN,Cortas,NaN
1,Cachorro,Mediano,NaN,Bicolor,Blanco,Marron,NaN,NaN,NaN,Largo,NaN,Cortas,Caidas
2,Adulto,Mediano,NaN,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Largo,NaN,Largas,NaN
3,Adulto,Mediano,NaN,NaN,NaN,NaN,NaN,Corto,NaN,Largo,NaN,NaN,Paradas
4,NaN,Chico,NaN,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Corto,NaN,Cortas,Caidas
5,NaN,Chico,NaN,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Corto,NaN,Cortas,Caidas
6,NaN,Mediano,NaN,NaN,Negro,NaN,NaN,Corto,Marron oscuro,Corto,NaN,Cortas,NaN
7,Adulto,Mediano,NaN,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Largo,Largo,NaN,Caidas
8,Cachorro,Chico,NaN,Bicolor,Blanco,NaN,NaN,Corto,NaN,NaN,Corto,Cortas,Caidas
9,Adulto,Grande,NaN,Liso,NaN,NaN,NaN,Mediano,Marron oscuro,Largo,Largo,Largas,NaN


### Imprimo los tipos de cada columna

In [22]:
print(prepared_data_train.dtypes)


Edad                 object
Tamaño               object
Sexo                 object
Patron de pelaje     object
Color de pelaje 1    object
Color de pelaje 2    object
Color de pelaje 3    object
Largo de pelaje      object
Color de ojos        object
Largo de hocico      object
Largo de cola        object
Largo de orejas      object
Tipo de orejas       object
dtype: object


### Obtengo la lista de etiquetas de las categorias

In [23]:
dataset_labels = prepared_data_train.columns.to_list()
dataset_labels

['Edad',
 'Tamaño',
 'Sexo',
 'Patron de pelaje',
 'Color de pelaje 1',
 'Color de pelaje 2',
 'Color de pelaje 3',
 'Largo de pelaje',
 'Color de ojos',
 'Largo de hocico',
 'Largo de cola',
 'Largo de orejas',
 'Tipo de orejas']

In [24]:
data_train_mascota_column = data_train.Mascota
data_train_mascota_column.head()

0    1
1    2
2    3
3    4
4    5
Name: Mascota, dtype: int64

Obtengo las categorias en base a las que no son numericas, en este caso todas son categorias

In [25]:
categorical_features_indices = np.where(prepared_data_train.dtypes != np.float)[0]
categorical_features_indices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

# Preparando el dataset de testeo

In [48]:
dataset_test = pandas.read_csv('./data/perrosTestDataAll.csv')
dataset_test.head()

,Mascota,Edad,Tamaño,Sexo,Patron de pelaje,Color de pelaje 1,Color de pelaje 2,Color de pelaje 3,Largo de pelaje,Color de ojos,Largo de hocico,Largo de cola,Largo de orejas,Tipo de orejas
0,4,Adulto,Grande,Hembra,Liso,Marron,NaN,NaN,Corto,Marron oscuro,Largo,Largo,Mediano,Paradas
1,16,Adulto,Chico,Macho,Bicolor,Marron,Negro,NaN,Mediano,Marron oscuro,Corto,Corto,Corto,Paradas
2,9,Cachorro,Chico,NaN,Bicolor,Blanco,NaN,NaN,Corto,NaN,NaN,Corto,Cortas,Caidas


In [49]:
null_value_stats = dataset_test.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Sexo                 1
Color de pelaje 2    2
Color de pelaje 3    3
Color de ojos        1
Largo de hocico      1
dtype: int64

In [50]:
dataset_test.fillna("NaN", inplace=True)
prepared_dataset_test = dataset_test.drop('Mascota', axis=1)
prepared_dataset_test.head()

,Edad,Tamaño,Sexo,Patron de pelaje,Color de pelaje 1,Color de pelaje 2,Color de pelaje 3,Largo de pelaje,Color de ojos,Largo de hocico,Largo de cola,Largo de orejas,Tipo de orejas
0,Adulto,Grande,Hembra,Liso,Marron,NaN,NaN,Corto,Marron oscuro,Largo,Largo,Mediano,Paradas
1,Adulto,Chico,Macho,Bicolor,Marron,Negro,NaN,Mediano,Marron oscuro,Corto,Corto,Corto,Paradas
2,Cachorro,Chico,NaN,Bicolor,Blanco,NaN,NaN,Corto,NaN,NaN,Corto,Cortas,Caidas


In [51]:
print(prepared_dataset_test.dtypes)

Edad                 object
Tamaño               object
Sexo                 object
Patron de pelaje     object
Color de pelaje 1    object
Color de pelaje 2    object
Color de pelaje 3    object
Largo de pelaje      object
Color de ojos        object
Largo de hocico      object
Largo de cola        object
Largo de orejas      object
Tipo de orejas       object
dtype: object


In [52]:
dataset_test_mascota_column = dataset_test.Mascota
dataset_test_mascota_column.head()

0     4
1    16
2     9
Name: Mascota, dtype: int64

# Creando y entrenando el modelo

### Creacion del modelo

In [53]:
eval_dataset = Pool(data=prepared_dataset_test,
                    label=dataset_test_mascota_column,
                    cat_features=categorical_features_indices)

In [58]:
train_dataset = Pool(data=prepared_data_train,
                        label=data_train_mascota_column,
                        cat_features=categorical_features_indices)

# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=1000,
                            learning_rate=1,
                            depth=5,
                            loss_function='MultiClass')
# Fit model

### Entrenamiento

In [59]:
model.fit(train_dataset)


0:	learn: 3.1326313	total: 79.7ms	remaining: 1m 19s
1:	learn: 2.8399750	total: 326ms	remaining: 2m 42s
2:	learn: 2.6750906	total: 636ms	remaining: 3m 31s
3:	learn: 2.4938895	total: 735ms	remaining: 3m 3s
4:	learn: 2.2585422	total: 1.08s	remaining: 3m 35s
5:	learn: 2.0395610	total: 1.38s	remaining: 3m 48s
6:	learn: 1.9433741	total: 1.69s	remaining: 3m 59s
7:	learn: 1.7723055	total: 2.01s	remaining: 4m 9s
8:	learn: 1.5917711	total: 2.28s	remaining: 4m 10s
9:	learn: 1.4266755	total: 2.5s	remaining: 4m 8s
10:	learn: 1.3804444	total: 2.82s	remaining: 4m 13s
11:	learn: 1.2725092	total: 3.09s	remaining: 4m 14s
12:	learn: 1.1579400	total: 3.28s	remaining: 4m 9s
13:	learn: 1.0892704	total: 3.5s	remaining: 4m 6s
14:	learn: 1.0122719	total: 3.67s	remaining: 4m 1s
15:	learn: 0.9109881	total: 3.9s	remaining: 3m 59s
16:	learn: 0.8688988	total: 4.12s	remaining: 3m 58s
17:	learn: 0.7936213	total: 4.44s	remaining: 4m 2s
18:	learn: 0.7526793	total: 4.63s	remaining: 3m 59s
19:	learn: 0.7048985	total: 4.8

Generar predicciones

In [60]:
# Get predicted classes
preds_class = model.predict(eval_dataset)
# Get predicted probabilities for each class
preds_proba = model.predict_proba(eval_dataset)

In [61]:
print(preds_class)

[[20]
 [20]
 [11]]


In [62]:
print(preds_proba)

[[0.02584631 0.02491893 0.0105377  0.01780486 0.00628558 0.06066528
  0.02161874 0.01303812 0.05054019 0.0211945  0.02656    0.01345411
  0.02819931 0.01488601 0.0078992  0.02237273 0.01225141 0.01108863
  0.15584275 0.341367   0.01257038 0.01691378 0.0137009  0.01799067
  0.05245289]
 [0.05640845 0.04069802 0.01784038 0.07153764 0.01460306 0.01361284
  0.02269794 0.00991919 0.06925968 0.0641533  0.03329349 0.01589616
  0.06847212 0.0212785  0.01660895 0.04677064 0.01108073 0.0170756
  0.04737941 0.14626827 0.01024323 0.07381702 0.0317244  0.04003433
  0.03932664]
 [0.06693073 0.01111363 0.00280826 0.0048153  0.02843394 0.09690878
  0.0259287  0.00339918 0.00382495 0.00524817 0.3531941  0.01086558
  0.01876711 0.00634353 0.02431882 0.25834692 0.00428673 0.00612228
  0.00598771 0.01677918 0.00978806 0.00754563 0.0110873  0.01255439
  0.00460101]]


### Supuestas predicciones

In [63]:
mascota_numero = 0

for prediccion in preds_class:
    probabilidad = math.trunc((preds_proba[mascota_numero][prediccion.item()-1])*100)
    print("-- Mascota "+str(dataset_test_mascota_column[mascota_numero])+" es similar a la mascota "+str(prediccion.item())
        +" => "+str(probabilidad)+"% de probabilidad")
    # print(str(preds_proba[mascota_numero]))
    # print(dataset_test.loc[mascota_numero:mascota_numero, "Edad":"Tipo de orejas"].values)
    # print(prepared_data_train.loc[prediccion.item():prediccion.item(), "Edad":"Tipo de orejas"].values)
    mascota_numero+=1


-- Mascota 4 es similar a la mascota 20 => 34% de probabilidad
-- Mascota 16 es similar a la mascota 20 => 14% de probabilidad
-- Mascota 9 es similar a la mascota 11 => 35% de probabilidad
